In [14]:
import json
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapely
import geopandas as gpd
import difflib
from geopy import distance



On charge le dataset de valeurs

In [2]:
dataset_df = pd.read_csv("construction_dataset/dataset6.csv", sep = ",")


In [4]:
selecja=dataset_df[['Code INSEE','Superficie','Population']]

['Unnamed: 0.1',
 'Unnamed: 0',
 'Code INSEE',
 'Code Postal',
 'Commune',
 'Département',
 'Région',
 'Statut',
 'Altitude Moyenne',
 'Superficie',
 'Population',
 'geo_point_2d',
 'geo_shape',
 'ID Geofla',
 'Code Commune',
 'Code Canton',
 'Code Arrondissement',
 'Code Département',
 'Code Région',
 'Commune_index',
 'code_index',
 'PrixMoyen_M2',
 'POPULATION',
 'Prix_moyen_exact',
 'soleil',
 '2G',
 '3G',
 '4G',
 '5G',
 'Total',
 'Pourcentage de gens éligible au debit :0,5 Mbit/s',
 'Pourcentage de gens éligible au debit :3 Mbit/s',
 'Pourcentage de gens éligible au debit :8 Mbit/s',
 'Pourcentage de gens éligible au debit :30 Mbit/s',
 'Pourcentage de gens éligible au debit :100 Mbit/s',
 'Pourcentage de gens éligible au debit :1 Gbit/s',
 'Fibre',
 'Câble',
 'DSL',
 'THD Radio',
 '4G Fixe',
 'HD Radio',
 'Satellite',
 'Pluviometrie_annuelle_mm',
 'Temperature min journaliere moyenne',
 'Temperature max journaliere moyenne',
 'Temperature moyenne journaliere moyenne',
 "('Code du

On commence par un input, bon ou mauvais

In [4]:
input_ville_variable="Tassin-la-demi-lune"
difflib.get_close_matches(input_ville_variable,dataset_df[~dataset_df['nom commune'].isna()]['nom commune'],n=10)

['Tassin-la-Demi-Lune',
 'Saint-Clair-de-Halouze',
 'Saint-Félix-de-Lunel',
 'Vaison-la-Romaine']

In [97]:
#on prend une des villes : maintenant elle est correcte
input_ville="KRIEGSHEIM"
result=dataset_df[dataset_df['Commune']==input_ville]

#on est censés obtenir une seule ligne désormais

In [74]:
dataset_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Code INSEE', 'Code Postal', 'Commune',
       'Département', 'Région', 'Statut', 'Altitude Moyenne', 'Superficie',
       'Population', 'geo_point_2d', 'geo_shape', 'ID Geofla', 'Code Commune',
       'Code Canton', 'Code Arrondissement', 'Code Département', 'Code Région',
       'Commune_index', 'code_index', 'PrixMoyen_M2', 'POPULATION',
       'Prix_moyen_exact', 'soleil', '2G', '3G', '4G', '5G', 'Total',
       'Pourcentage de gens éligible au debit :0,5 Mbit/s',
       'Pourcentage de gens éligible au debit :3 Mbit/s',
       'Pourcentage de gens éligible au debit :8 Mbit/s',
       'Pourcentage de gens éligible au debit :30 Mbit/s',
       'Pourcentage de gens éligible au debit :100 Mbit/s',
       'Pourcentage de gens éligible au debit :1 Gbit/s', 'Fibre', 'Câble',
       'DSL', 'THD Radio', '4G Fixe', 'HD Radio', 'Satellite',
       'Pluviometrie_annuelle_mm', 'Temperature min journaliere moyenne',
       'Temperature max journaliere moyenn

In [85]:
result

,Unnamed: 0.1,Unnamed: 0,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,...,"('Libellé de la commune', 'ENS')","('Libellé de la commune', 'LR')","('Libellé de la commune', 'NUP')","('Libellé de la commune', 'REG')","('Libellé de la commune', 'UDI')",nom commune,APL aux generalistes,APL aux généralistes de moins de 65 ans,population normalisee par generaliste,densite
3708,3708,3708,93079,93430,VILLETANEUSE,['SEINE-SAINT-DENIS'],['ILE-DE-FRANCE'],['Commune simple'],43.0,232.0,...,0.0,0.0,1.0,0.0,0.0,Villetaneuse,"4,7","4,3",10 827,0.055172


In [78]:
int(result['Code Département'])

1

In [39]:
#pour les critères
criteres=['Département','Région','Superficie','Population','PrixMoyen_M2','soleil','Pluviometrie_annuelle_mm','Temperature min journaliere moyenne','Temperature max journaliere moyenne','Temperature moyenne journaliere moyenne',' APL aux generalistes']

dico_resultat={}
for x in criteres :
    dico_resultat[x]=result[x].iloc[0]

couleur_politique_ville=''
for x in ["('Libellé de la commune', 'DVD')",
       "('Libellé de la commune', 'DVG')", "('Libellé de la commune', 'ENS')",
       "('Libellé de la commune', 'LR')", "('Libellé de la commune', 'NUP')",
       "('Libellé de la commune', 'REG')", "('Libellé de la commune', 'UDI')"]:
    if result[x].iloc[0]==1:
        if len(couleur_politique_ville)==0:
            couleur_politique_ville=couleur_politique_ville+x[27:30]
        else:
            couleur_politique_ville=couleur_politique_ville+"/"+x[27:30]
dico_resultat['politique_ville']=couleur_politique_ville

couleur_politique_dep=''
for x in ["('Code du département', 'DVD')", "('Code du département', 'DVG')",
       "('Code du département', 'ENS')", "('Code du département', 'LR')",
       "('Code du département', 'NUP')", "('Code du département', 'REG')",
      "('Code du département', 'UDI')", "('Libellé de la commune', 'DVD')"]:
    if result[x].iloc[0]==1:
        if len(couleur_politique_dep)==0:
            couleur_politique_dep=couleur_politique_dep+x[25:28]
        else:
            couleur_politique_dep=couleur_politique_dep+"/"+x[25:28]
dico_resultat['politique_dep']=couleur_politique_dep

dico_resultat['reseau']=str(int(result['Total'].iloc[0]))+" ("+str(result[['2G', '3G', '4G', '5G']].idxmax(axis='columns').iloc[0])+")"

dico_resultat

{'Département': "['RHONE']",
 'Région': "['RHONE-ALPES']",
 'Superficie': 795.0,
 'Population': 19.0,
 'PrixMoyen_M2': 3774.0,
 'soleil': 183,
 'Pluviometrie_annuelle_mm': 660,
 'Temperature min journaliere moyenne': 9.270947765212709,
 'Temperature max journaliere moyenne': 18.47347872913301,
 'Temperature moyenne journaliere moyenne': 13.87237479806139,
 ' APL aux generalistes': '  5,3   ',
 'politique_ville': 'NUP',
 'politique_dep': 'NUP',
 'reseau': '157 (4G)'}

dessin sur la carte de la ville

In [7]:
# pour le dessin : geo_shape
shape_ville=result['geo_shape'].iloc[0]


pour le calcul des différentes distances

In [98]:
loc_ville=result['geo_point_2d'].iloc[0]

In [66]:
#rappel pour le calcul des distances

paris=[48.866667,2.333333]
evreux=[49.023 ,1.154 ]

coords_1 = paris
coords_2 = evreux

print(geopy.distance.geodesic(coords_1, coords_2).km)


def calcul(ville,l1,l2):
    try:
        return geopy.distance.geodesic(ville, [l1,l2]).km
    except:
        return 10000


88.1202315083897


sortie d'autoroute la plus proche

In [64]:
autoroutes=pd.read_csv("datasets_proximite/autoroutes.csv", sep = ";")
autoroutes['distance']=autoroutes.apply(lambda x: calcul(evreux,x.long,x.lat),axis=1)
autoroutes[autoroutes['distance']==autoroutes['distance'].min()]

,Numero de l'autoroute,Connexion paris,Numéro de la sortie,Coordonnées géo,Unnamed: 4,long,lat,c_long,c_lat,vraie_distance(km),duree(minutes),distance
219,Autoroute A13,"48.84828008728176, 2.249221937847407",17,"49.13519111674638, 1.312927650160255",NaN,49.135191,1.312928,48.84828,2.249222,89.4855,54.288333,17.043448


hopital le plus proche

In [67]:
hopitaux=pd.read_csv("datasets_proximite/hopitaux_update.csv", sep = ",")
hopitaux['distance']=hopitaux.apply(lambda x: calcul(evreux,x.long,x.lat),axis=1)
hopitaux[hopitaux['distance']==hopitaux['distance'].min()]

,Unnamed: 0,Geo Point,Nom,"Public (hôpital), privé (clinique)",Présence d'un service d'urgences,Accessible PMR,Site web,Téléphone,Email,Exploitant,...,type:FR:FINESS,ref:FR:FINESS,Élément OSM,Élément Wikipédia,Élément Wikidata,Source,other_tags,long,lat,distance
400,400,"49.0195223, 1.145894",La Clinique Bergouignan,clinic,NaN,NaN,http://clinique-evreux-hospitalisation-eure-ca...,+33 2 32 31 45 25,NaN,NaN,...,NaN,NaN,https://www.openstreetmap.org/api/0.6/node/368...,NaN,NaN,NaN,"{""addr:housenumber"": ""1"", ""addr:postcode"": ""27...",49.019522,1.145894,0.707873


In [69]:
hopitaux.apply(lambda x: calcul(evreux,x.long,x.lat),axis=1).min()

0.7078730351972777

ecole la plus proche selon le type

limiter la recherche aux départements limitrophes

In [73]:
ecoles=pd.read_csv("datasets_proximite/ecoles_update.csv", sep = ";")
ecoles=ecoles[~ecoles['Latitude'].isna()]
ecoles=ecoles[~ecoles['Longitude'].isna()]
ecoles['distance']=ecoles.apply(lambda x: calcul(evreux,x.Latitude,x.Longitude),axis=1)
ecoles[ecoles['distance']==ecoles[ecoles["lycee_general"]==1]["distance"].min()]

,Unnamed: 0,Code établissement,Appellation officielle,Dénomination principale,Patronyme uai,Secteur Public/Privé,Adresse,Lieu dit,Boite postale,Code postal,...,code_ministere,libelle_ministere,date_ouverture,lycee_general,lycee_techno,maternelle,college,primaire,specialise,distance
22495,22495,0270016W,Lycée général et technologique Aristide Briand,LYCEE GENERAL ET TECHNOLOGIQUE,ARISTIDE BRIAND,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,01/05/1965,1,1,0,0,0,0,0.719585
22567,22567,0271268G,Lycée professionnel Aristide Briand,LYCEE PROFESSIONNEL,ARISTIDE BRIAND,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,29/03/1973,0,1,0,0,0,0,0.719585
48930,48930,0271870L,Micro-lycée de l'académie de Rouen,ETABLISSEMENT EXPERIMENTAL,MICRO-LYCEE ACADEMIE DE ROUEN,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,01/09/2014,1,1,0,0,0,0,0.719585


In [77]:
deps=list(ecoles['Code département'].unique())
ecoles_a_examiner=ecoles
for x in deps:
    if calcul(evreux,ecoles[ecoles['Code département']==x]['Latitude'].iloc[0],ecoles[ecoles['Code département']==x]['Longitude'].iloc[0])>50:
        ecoles_a_examiner=ecoles_a_examiner[ecoles_a_examiner['Code département']!=x]
ecoles_a_examiner['distance']=ecoles_a_examiner.apply(lambda x: calcul(evreux,x.Latitude,x.Longitude),axis=1)
ecoles_a_examiner[ecoles_a_examiner['distance']==ecoles_a_examiner[ecoles_a_examiner["lycee_general"]==1]["distance"].min()]

,Unnamed: 0,Code établissement,Appellation officielle,Dénomination principale,Patronyme uai,Secteur Public/Privé,Adresse,Lieu dit,Boite postale,Code postal,...,code_ministere,libelle_ministere,date_ouverture,lycee_general,lycee_techno,maternelle,college,primaire,specialise,distance
22495,22495,0270016W,Lycée général et technologique Aristide Briand,LYCEE GENERAL ET TECHNOLOGIQUE,ARISTIDE BRIAND,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,01/05/1965,1,1,0,0,0,0,0.719585
22567,22567,0271268G,Lycée professionnel Aristide Briand,LYCEE PROFESSIONNEL,ARISTIDE BRIAND,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,29/03/1973,0,1,0,0,0,0,0.719585
48930,48930,0271870L,Micro-lycée de l'académie de Rouen,ETABLISSEMENT EXPERIMENTAL,MICRO-LYCEE ACADEMIE DE ROUEN,Public,2 rue Pierre Semard,NaN,3126,27031,...,6,MINISTERE DE L'EDUCATION NATIONALE,01/09/2014,1,1,0,0,0,0,0.719585


avec les voisins (10 fois plus rapide) :

In [103]:
code_departement="67"
voisins=pd.read_csv("datasets_proximite/voisins_update.csv")
ecoles=pd.read_csv("datasets_proximite/ecoles_update.csv", sep = ";")
ecoles=ecoles[~ecoles['Latitude'].isna()]
ecoles=ecoles[~ecoles['Longitude'].isna()]
voisins_interet=list(voisins[voisins['code_dep']==float(code_departement)]["codes_voisins"].iloc[0].split(","))
voisins_interet.append(code_departement)
ecoles_a_examiner=ecoles[ecoles['Code département'].isin(voisins_interet)]
ecoles_a_examiner['distance']=ecoles_a_examiner.apply(lambda x: calcul(loc_ville,x.Latitude,x.Longitude),axis=1)

types=["lycee_general","lycee_techno","maternelle","college","primaire","specialise"]

dico_ecoles={}
for x in types :
    df=ecoles_a_examiner[ecoles_a_examiner[x]==1]
    dico_ecoles[x]=df['distance'].min()
dico_ecoles

mine=dico_ecoles['lycee_general']
maxe=dico_ecoles['lycee_general']
for x in dico_ecoles.keys():
    if dico_ecoles[x]<mine:
        mine=dico_ecoles[x]
    elif dico_ecoles[x]>maxe:
        maxe=dico_ecoles[x]
    else:
        pass

(mine,maxe)

C:\Users\USER\AppData\Local\Temp\ipykernel_19212\2002170958.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecoles_a_examiner['distance']=ecoles_a_examiner.apply(lambda x: calcul(loc_ville,x.Latitude,x.Longitude),axis=1)


(1.066274418644135, 5.440844075786284)

In [ ]:
min=dico_ecoles['lycee_general']
max=dico_ecoles['lycee_general']
for x in dico_ecoles.keys():
    if dico_ecoles[x]<min:
        min=dico_ecoles[x]
    elif dico_ecoles[x]>max:
        max=dico_ecoles[x]
    else:
        pass




littoraux

In [81]:
def read_list(name):
    # for reading also binary mode is important
    with open("datasets_proximite/littoral_"+name+'.json', 'rb') as fp:
        n_list = json.load(fp)
        return n_list
    
littoral_med=read_list("med")
littoral_atlantique=read_list("atlantique")

In [84]:
min_med=geopy.distance.geodesic(evreux, littoral_med[0]).km
min_atl=geopy.distance.geodesic(evreux, littoral_atlantique[0]).km

for x in littoral_med:
    cand=geopy.distance.geodesic(evreux, x).km
    if cand<min_med:
        min_med=cand

for x in littoral_atlantique:
    cand=geopy.distance.geodesic(evreux, x).km
    if cand<min_atl:
        min_atl=cand

if min_med<min_atl:
    print("med")
else:
    print('atl')
print(min(min_med,min_atl))

atl
74.84970227125451


In [11]:
dataset_df['densite']=dataset_df['Population']/dataset_df['Superficie']

In [3]:
dataset_df = pd.read_csv("construction_dataset/dataset6.csv", sep = ",")
dataset_df['densite']=dataset_df['Population']/dataset_df['Superficie']

In [80]:
dataset_df[' APL aux generalistes']=dataset_df[' APL aux generalistes'].str.replace(r'[^\d,]', '')
dataset_df[' APL aux generalistes']=dataset_df[' APL aux generalistes'].str.replace(',', '.')
dataset_df[' APL aux generalistes']=pd.to_numeric(dataset_df[' APL aux generalistes'])

C:\Users\USER\AppData\Local\Temp\ipykernel_29268\1469305374.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset_df[' APL aux generalistes']=dataset_df[' APL aux generalistes'].str.replace(r'[^\d,]', '')


In [83]:
ecremage=[]

##densite
a=dataset_df['densite'].median()
b=dataset_df['densite'].std()
selected=dataset_df[dataset_df['densite']<0.5*a]
ecremage.append(selected.shape[0])

##couleur politique
selected=selected[selected["('Libellé de la commune', 'NUP')"]==1]
ecremage.append(selected.shape[0])

##4G dispo
selected=selected[selected["Fibre"]!="0"]
ecremage.append(selected.shape[0])

##fibre dispo
selected=selected[selected["4G"]>6]
ecremage.append(selected.shape[0])

##soleil
selected=selected[selected["soleil"]>100]
ecremage.append(selected.shape[0])

##soleil
selected=selected[selected['Superficie']>2000]
ecremage.append(selected.shape[0])

##soleil
selected=selected[dataset_df[' APL aux generalistes']>5]
ecremage.append(selected.shape[0])


ecremage

C:\Users\USER\AppData\Local\Temp\ipykernel_29268\912604388.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  selected=selected[dataset_df[' APL aux generalistes']>5]


[9743, 4640, 1865, 529, 529, 343, 31]

In [97]:
result=selected["geo_shape"]
with open("example.json", 'w',encoding="ascii") as f:
    json.dump(result.to_json(), f)

In [84]:

xs, ys = list(), list()

erreur=0
for k in range(len(result)):
    commune_shape = eval(result.iloc[k])
    try : 
        # Some cities have two zones, hence np.shape will yield (non-homogeneous array).
        commune_coords = np.asarray(commune_shape["coordinates"])[0]
        commune_coords = np.concatenate([commune_coords, commune_coords[0].reshape(1, 2)])
        xs_, ys_ = zip(*commune_coords)
        xs.append(xs_)
        ys.append(ys_)
    except:
        try:    
            # If the commune has several zones, iterating over them. Each zone is of the shape (1, x, y) - do not ask me why though
            commune_coords = [np.asarray(c_[0]) for c_ in commune_shape["coordinates"]]
            for i, zone in enumerate(commune_coords):
                commune_coords[i] = np.concatenate([commune_coords[i], commune_coords[i][0].reshape(1, 2)])
                xs_, ys_ = zip(*commune_coords[i])
                xs.append(xs_)
                ys.append(ys_)
        except:
            erreur+=1
        

# Create a map anywhere near a point, and define the contours as a shapely Polygon object - handy to manipulate.
m = folium.Map(location = [48,2], zoom_start=10, tiles='OpenStreetMap')
crs = {'init': 'epsg:4326'}

limite_affichage=len(result)+1 #limite de 100 = ~4s
affiche=0
for (x_, y_) in zip(xs, ys):
    affiche+=1
    if affiche>limite_affichage:
        break
    polygon_geom = shapely.geometry.Polygon(zip(x_, y_))
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])

    # You can edit style - details in https://leafletjs.com/reference.html#path-option
    styl = {'fillColor': "#FF0000", 'color':'#FF0000', 'weight':3, "opacity": 1, "fillOpacity": .3}
    folium.GeoJson(polygon, style_function = lambda y: styl).add_to(m)

# The elegant and sober Python functionality of randomly writing "m" to display a map
m 

C:\Users\USER\AppData\Local\Temp\ipykernel_29268\1090543700.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  commune_coords = np.asarray(commune_shape["coordinates"])[0]
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

## Gradient zone

In [1]:
import json
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapely
import geopandas as gpd
import difflib
import geopy.distance

In [2]:
dataset_df = pd.read_csv("construction_dataset/dataset6.csv", sep = ",")

dataset_df[["lat","long"]]=dataset_df["geo_point_2d"].str.split(",",expand=True)
dataset_df["lat"]=pd.to_numeric(dataset_df["lat"])
dataset_df["long"]=pd.to_numeric(dataset_df["long"])

In [3]:
selected=dataset_df[dataset_df["lat"]<48.68]
selected=selected[selected["lat"]>48.37]
selected=selected[selected["long"]>-0.62]
selected=selected[selected["long"]<-0.22]

In [4]:
def rgb_to_hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

In [30]:
result=selected["geo_shape"]
with open("example2.json", 'w',encoding="ascii") as f:
    json.dump(result.to_json(), f)

In [33]:
result=selected["geo_shape"]
result2=selected[["lat","long"]]

xs, ys = list(), list()
dist_liste=[]
erreur=0
for k in range(len(result)):
    commune_shape = eval(result.iloc[k])
    try : 
        # Some cities have two zones, hence np.shape will yield (non-homogeneous array).
        commune_coords = np.asarray(commune_shape["coordinates"])[0]
        commune_coords = np.concatenate([commune_coords, commune_coords[0].reshape(1, 2)])
        xs_, ys_ = zip(*commune_coords)
        xs.append(xs_)
        ys.append(ys_)
        dist=((result2["lat"].iloc[k]-48.53)**2+(result2["long"].iloc[k]+0.42)**2)
        dist_liste.append(dist)
    except:
        try:    
            # If the commune has several zones, iterating over them. Each zone is of the shape (1, x, y) - do not ask me why though
            commune_coords = [np.asarray(c_[0]) for c_ in commune_shape["coordinates"]]
            for i, zone in enumerate(commune_coords):
                commune_coords[i] = np.concatenate([commune_coords[i], commune_coords[i][0].reshape(1, 2)])
                xs_, ys_ = zip(*commune_coords[i])
                xs.append(xs_)
                ys.append(ys_)
            dist=((result2["lat"].iloc[k]-48.53)**2+(result2["long"].iloc[k]+0.42)**2)
            dist_liste.append(dist)
        except:
            erreur+=1
            dist_liste.append(0)
        

# Create a map anywhere near a point, and define the contours as a shapely Polygon object - handy to manipulate.
m = folium.Map(location = [48.53,-0.42], zoom_start=10, tiles='OpenStreetMap')
crs = {'init': 'epsg:4326'}

limite_affichage=len(result)+1 #limite de 100 = ~4s
affiche=0
maximum=max(dist_liste)
dist_liste2=[]
for k in dist_liste:
    dist_liste2.append(k/maximum)

print(maximum)

0.04838989448730441


In [29]:
p=0
liste_poly=[]
for (x_, y_) in zip(xs, ys):
    affiche+=1
    if affiche>limite_affichage:
        break
    polygon_geom = shapely.geometry.Polygon(zip(x_, y_))
    liste_poly.append(gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]))

    # You can edit style - details in https://leafletjs.com/reference.html#path-option
    if dist_liste2[p]<0.5:
        couleur=rgb_to_hex(74+int(dist_liste2[p]*2*125), 199, 72)
    else:
        couleur=rgb_to_hex(199, int(199-((dist_liste2[p]-0.5)*2*125)), 72)
    folium.GeoJson(liste_poly[p], style_function = lambda y: {'fillColor': couleur, 'color':'#FF0000', "fillOpacity": 0.7, 'weight':1, "opacity": 1}).add_to(m)
    p+=1

# The elegant and sober Python functionality of randomly writing "m" to display a map
m 

## Création dataset complet

In [47]:
## merged2_df.to_csv('dataset6.csv')  
dataset_df = pd.read_csv("construction_dataset/randomized_dataset.csv", sep = ",")
voisins=pd.read_csv("datasets_proximite/voisins_update.csv")
ecoles=pd.read_csv("datasets_proximite/ecoles_update.csv", sep = ";")

##
dataset_df[['Latitude', 'Longitude']]=dataset_df['geo_point_2d'].str.split(',',expand=True)

##traitement du dataset ecoles
ecoles=ecoles[~ecoles['Latitude'].isna()]
ecoles=ecoles[~ecoles['Longitude'].isna()]
types=["lycee_general","lycee_techno","maternelle","college","primaire","specialise"]

##fonction distance
def calcul(ville,l1,l2):
    try:
        return distance.geodesic(ville, [l1,l2]).km
    except:
        return 10000
    

In [48]:
import geopy

for villes in dataset_df.index:
    print(villes)
    for x in types:
        distances = []
        codedep=dataset_df.loc[villes,"Code Département"]
        voisins_interet=list(voisins[voisins['code_dep']==float(codedep)]["codes_voisins"].iloc[0].split(","))
        voisins_interet.append(str(codedep))
        ecoles2=ecoles[ecoles[x]==1]
        ecoles2=ecoles2[ecoles2["Code département"].isin(voisins_interet)]
        for eco in ecoles2.index:
            distances.append(geopy.distance.distance(dataset_df.loc[villes, ['Latitude', 'Longitude']], ecoles2.loc[eco,['Latitude', 'Longitude']]))
        min_dist = min(distances)
        dataset_df.loc[villes, 'min_distance_to_'+str(x)] = min_dist


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


KeyboardInterrupt: 

In [ ]:
dataset_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,...,random_key,nom-presente,Latitude,Longitude,min_distance_to_lycee_general,min_distance_to_lycee_techno,min_distance_to_maternelle,min_distance_to_college,min_distance_to_primaire,min_distance_to_specialise
44,44,44,44,9145,09100,LES ISSARDS,['ARIEGE'],['MIDI-PYRENEES'],['Commune simple'],300.0,...,nITjxD0pBv,LES ISSARDS (9),43.07948607186958,1.739013767517685,9.198716698489891 km,9.198716698489891 km,3.597783543843029 km,10.302297296229963 km,1.6723240077781285 km,10.268695662224381 km
48,48,48,48,3076,03230,CHEZY,['ALLIER'],['AUVERGNE'],['Commune simple'],248.0,...,DeGtNB3ut5,CHEZY (3),46.61247575199707,3.459348047973815,9.305210537908975 km,9.305210537908975 km,3.4306690951683447 km,9.919717354113395 km,NaN,NaN
54,54,54,54,8202,08230,GUE-D'HOSSUS,['ARDENNES'],['CHAMPAGNE-ARDENNE'],['Commune simple'],336.0,...,8ojSCrpZnf,GUE-D'HOSSUS (8),49.953460075082134,4.539012965510342,NaN,NaN,NaN,NaN,NaN,NaN
59,59,59,59,8154,08260,ESTREBAY,['ARDENNES'],['CHAMPAGNE-ARDENNE'],['Commune simple'],249.0,...,bWJOPVkgSt,ESTREBAY (8),49.82534400876408,4.345687742972739,NaN,NaN,NaN,NaN,NaN,NaN
77,77,77,77,4233,04200,VAUMEILH,['ALPES-DE-HAUTE-PROVENCE'],"[""PROVENCE-ALPES-COTE D'AZUR""]",['Commune simple'],658.0,...,7MwWBJvzYy,VAUMEILH (4),44.290307695711675,5.956531218170149,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36196,36196,36196,36196,1314,01160,PRIAY,['AIN'],['RHONE-ALPES'],['Commune simple'],284.0,...,E3kPwNyocp,PRIAY (1),46.019752148615915,5.285233538031973,NaN,NaN,NaN,NaN,NaN,NaN
36197,36197,36197,36197,2624,02120,PROISY,['AISNE'],['PICARDIE'],['Commune simple'],150.0,...,Dn19rj7YfV,PROISY (2),49.89061724844165,3.752411209474499,NaN,NaN,NaN,NaN,NaN,NaN
36199,36199,36199,36199,2815,02590,VILLERS-SAINT-CHRISTOPHE,['AISNE'],['PICARDIE'],['Commune simple'],83.0,...,JhtCvKV6BO,VILLERS-SAINT-CHRISTOPHE (2),49.77969141553496,3.090998974963444,NaN,NaN,NaN,NaN,NaN,NaN
36230,36230,36230,36230,7269,07310,SAINT-MARTIN-DE-VALAMAS,['ARDECHE'],['RHONE-ALPES'],['Chef-lieu canton'],690.0,...,aueE0TOSwq,SAINT-MARTIN-DE-VALAMAS (7),44.93659102492989,4.370761691987222,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
##traitement du dataset voisins
voisins_interet=list(voisins[voisins['code_dep']==float(code_departement)]["codes_voisins"].iloc[0].split(","))
voisins_interet.append(code_departement)

##
ecoles_a_examiner=ecoles[ecoles['Code département'].isin(voisins_interet)]
ecoles_a_examiner['distance']=ecoles_a_examiner.apply(lambda x: calcul(loc_ville,x.Latitude,x.Longitude),axis=1)